## EDA

# Descripción general del chunk

Este bloque de código realiza una **exploración inicial de la base de datos SQLite** `imedia.sqlite`, con el propósito de identificar las tablas disponibles y determinar cuál de ellas es más relevante para el análisis y modelado posterior.

## Pasos realizados

1. **Conexión y exploración de la base de datos:**  
   Se conecta a `../db/imedia.sqlite` y se ejecuta una consulta a `sqlite_master` para listar todas las tablas existentes.  
   Se identificaron cuatro tablas principales: `subreddits`, `authors`, `posts` y `comments`.

2. **Carga de datos:**  
   Cada tabla fue leída mediante `pandas.read_sql_query` y almacenada en un DataFrame (`authors`, `comments`, `posts`, `subreddits`).

3. **Vista preliminar:**  
   Se revisaron las primeras filas de cada tabla para entender su estructura y verificar la calidad de los datos.

4. **Cierre de conexión:**  
   La conexión se cerró correctamente para liberar recursos.

## Justificación del enfoque en la tabla `posts`

Tras revisar la estructura de la base, se determinó que **la tabla `posts` es la más adecuada para realizar el análisis y modelado**, por las siguientes razones:

- **Centralidad del contenido:** cada fila representa una publicación única, que condensa información clave sobre la interacción y desempeño dentro de los subreddits.  
- **Variables predictoras ricas:** contiene métricas cuantitativas (`score`, `num_comments`, `created_utc`) y cualitativas (`title`, `selftext`, `link_flair_text`, `subreddit`, `author`) que permiten abordar tanto análisis descriptivos como modelos de predicción.  
- **Integración natural con otras tablas:** puede relacionarse fácilmente con `authors` y `subreddits` para enriquecer el análisis, sin perder su enfoque principal.  
- **Objetivo analítico claro:** al centrarnos en los posts, se pueden construir modelos orientados a explicar o predecir el rendimiento de una publicación (por ejemplo, su puntuación o nivel de interacción).

En síntesis, la tabla `posts` actúa como el **núcleo del ecosistema de datos**, integrando dimensiones de autor, comunidad y contenido, lo que la convierte en el punto de partida más lógico para el análisis exploratorio y la fase de modelado.

---

# EDA inicial de la tabla `posts`

Este bloque realiza un **Análisis Exploratorio de Datos (EDA)** sobre `posts` para revisar estructura, calidad y comportamiento de variables clave que servirán de base para el modelado.

## Descripción de las acciones

1. **Exploración general**
   - `posts` contiene **1542 registros y 16 columnas**.  
   - Tipos de datos: **9 `object`**, **6 `int64`**, **1 `float64`**.  
   - Columnas: `post_id`, `title`, `selftext`, `url`, `permalink`, `score`, `num_comments`, `over_18`, `created_utc`, `link_flair_text`, `is_self`, `spoiler`, `locked`, `thumbnail`, `subreddit`, `author`.

2. **Valores nulos**
   - Solo `link_flair_text` presenta **368 nulos**.  
   - El resto de columnas **no tiene** valores faltantes.  
   - La salida del código imprime la tabla ordenada de nulos para confirmarlo.

3. **Descripción de variables (resumen)**


| Tipo | Variable | Descripción | Observaciones |
|---|---|---|---|
| ID | `post_id` | Identificador único del post. | Útil para joins; se considera fuga en modelado. |
| Texto | `title` | Título del post. | Siempre presente. |
| Texto | `selftext` | Cuerpo del post (si aplica). | Frecuentemente vacío en posts de enlace. |
| URL | `url` | Enlace externo o a Reddit. | Presente en todos los registros. |
| URL | `permalink` | Enlace interno a Reddit. | Rutas dentro del subreddit. |
| Métrica | `score` | Puntuación/karma del post. | Distribución con cola larga (outliers). |
| Métrica | `num_comments` | Número de comentarios. | Alta variabilidad. |
| Binaria | `over_18` | 1 si es NSFW. | ~0.6% positivos. |
| Temporal | `created_utc` | Timestamp Unix de creación. | Convertible a fecha (`created_dt`) para análisis temporal. |
| Categórica | `link_flair_text` | Flair o categoría del post. | 368 nulos. |
| Binaria | `is_self` | 1 si es post de texto, 0 si es enlace. | ~**20%** son de texto. |
| Binaria | `spoiler` | Indica spoiler. | Poco frecuente. |
| Binaria | `locked` | Comentarios bloqueados. | 0 en todos los casos. |
| Texto | `thumbnail` | Tipo de miniatura. | Poca variabilidad. |
| Categórica | `subreddit` | Comunidad del post. | Predomina **r/Python**. |
| Texto | `author` | Usuario autor. | Muchos autores únicos. |

4. **Estadísticas descriptivas (numéricas)**
- `score` → **mean 935.04**, **std 4038.68**, **p50 22**, **p75 182**, **max 65830**.  
- `num_comments` → **mean 89.77**, **std 379.98**, **p50 7**, **p75 35**, **max 6411**.  
- Binarias (`is_self`, `spoiler`, `over_18`) muestran baja proporción de 1’s, excepto `is_self` (~0.20).

5. **Distribución de variables (gráficas)**
- Los histogramas de **`score`** y **`num_comments`** confirman **asimetría positiva marcada** (cola larga).  
- Implicación: conviene **clipping** o **transformación logarítmica** antes del modelado.

6. **Análisis categórico**
- **Top 10 flairs**: destacan *Cat Picture - OC*, *No Paywall*, *music*, *Possible Paywall*, *Discussion*, *discussion*, *Artwork*, *article*, *Baking Advice Needed*, *Question*.  
- Señal de mezcla entre contenido informativo, artístico y de entretenimiento.

7. **Tipo de publicación (`is_self`)**
- El conteo indica que **~80% son enlaces (is_self=0)** y **~20% son posts de texto (is_self=1)**.  
- La comunidad privilegia compartir recursos externos.

8. **Correlaciones numéricas**
- **`score` ↔ `num_comments`: r ≈ 0.45** (positiva moderada).  
- `is_self` muestra **relación negativa leve con `score` (≈ -0.09)** → posts de texto tienden a menor puntuación.  
- `created_utc` presenta correlaciones negativas débiles con `score` y `num_comments` (≈ -0.24 / -0.26).  
- No se observan **multicolinealidades fuertes**.

9. **Tendencia temporal**
- Con `created_dt` derivado de `created_utc`, la serie muestra **tendencia creciente** en el volumen de publicaciones de finales de septiembre a noviembre de 2025, con un **pico pronunciado** a mediados de noviembre.

## Insights clave

- `posts` es **completo y consistente**, adecuado para modelado.  
- **Alta dispersión y fuerte asimetría** en `score` y `num_comments` justifican **recorte de outliers** o **`log1p`**.  
- El **engagement** (comentarios) se asocia con mayor `score`, reforzando su rol como señal principal.  
- Los flairs más comunes revelan diversidad temática; la preferencia por **enlaces** sugiere que el contenido externo impulsa la participación.  
- La tendencia temporal al alza puede aprovecharse como **feature temporal** en modelos predictivos.

---

# EDA extendido: análisis del rendimiento de los posts (`score`)

Este bloque amplía el análisis del **target `score`** dentro de la tabla `posts`, explorando su comportamiento general, su relación con otras variables y su evolución temporal.  
El propósito es comprender los factores asociados al rendimiento o viralidad de las publicaciones.

---

## Información general del dataset

- Total de registros: **1542 publicaciones**  
- Total de variables: **17 columnas** (`created_dt` ya derivada)  
- Tipos de datos:  
  - 9 `object`, 6 `int64`, 1 `float64`, 1 `datetime64[ns]`  
- Valores nulos: **solo `link_flair_text` (368 faltantes)**  
- Variables binarias: `over_18`, `is_self`, `spoiler`, `locked`  
- Columnas potencialmente con fuga para modelado supervisado: `post_id`, `url`, `permalink`, `thumbnail`, `score`

---

## Distribución del target `score`

- **count 1542 • mean 935.04 • median 22 • p75 182 • max 65,830**  
- La distribución es **altamente asimétrica con cola derecha** (pocos posts muy virales).  
- La transformación **`log1p(score)`** revela una forma más estable para análisis y modelado.

---

## Relaciones con otras variables

### 1) `num_comments` vs `score` (escala log1p)
- Dispersión con **tendencia creciente clara**; correlación aproximada **r ≈ 0.45**.  
- Conclusión: **más comentarios ↔ mayor score**, aunque existen outliers.

### 2) Variables binarias (`is_self`, `over_18`, `spoiler`, `locked`)
- **`is_self`**: los **enlaces (0)** superan a los posts de **texto (1)** en score (boxplot).  
- **`over_18` y `spoiler`**: poco frecuentes; efecto marginal.  
- **`locked`**: sin variabilidad (todos 0).

### 3) `link_flair_text` y rendimiento promedio
- **Top por score promedio (ejemplos del Top 10)**: *Misleading Title*, *Release the Epstein files*, *Tylenol Rapid Release the Epstein files*, *I feel bad for the dog*, *Arts/Crafts*, *ADBLOCK WARNING*, *Image*, `:Misc:` y *AMA*.  
- Lectura: ciertos **tópicos polémicos/actualidad** elevan la visibilidad.

### 4) Evolución temporal del score
- El **score promedio diario decrece** a lo largo del periodo observado, con picos puntuales → efecto de **eventos coyunturales**.

### 5) Longitud de texto y título
- `title_len` y `selftext_len` muestran **relación débil** con `score`: hay posts de alto rendimiento en rangos de longitud variados.

### 6) Contenido textual (WordCloud)
- En títulos con alto score destacan términos como **“Trump”, “Democrat”, “shutdown”, “Supreme Court”, “election”, “cat”** → **política/actualidad** y temas virales.

---

## Insights principales

- **Alta asimetría del target**: pocos posts explican la mayor parte del score total.  
- **Engagement** (comentarios) es el principal correlato del éxito.  
- **Tipo de contenido**: los enlaces tienden a rendir mejor que los posts de texto.  
- **Temática**: flairs y keywords polémicas/actualidad concentran los mayores promedios de score.  
- **Temporalidad**: caída del promedio con picos event-driven.  
- **Longitud**: no determina el rendimiento.

---

## Recomendaciones y decisiones de EDA

1. **Usar `log1p(score)`** o un objetivo robusto (p. ej., `score_clipped`) para estabilizar la escala.  
2. **Excluir fugas**: `post_id`, `url`, `permalink`, `thumbnail` (y el propio `score` como feature).  
3. **Imputar nulos** en `link_flair_text` (p. ej., `'unknown'`) y **normalizar/collapse** categorías raras.  
4. **Features recomendadas**:  
   - **Numéricas**: `num_comments`, `title_len`, `selftext_len`  
   - **Categóricas**: `is_self`, `subreddit`, `author`, `link_flair_text`  
   - **Temporales** derivados de `created_dt` (mes, día de semana, hora, recency)  
5. **Validación temporal** cuando se modele por fecha (evitar fuga futura).

**Conclusión:** el rendimiento de un post se explica ante todo por **interacción y temática**; la **longitud** y el **formato de texto** por sí solos no anticipan el éxito.


In [2]:
import sqlite3
import pandas as pd


db_path = "../db/imedia.sqlite"
conn = sqlite3.connect(db_path)

# Mostrar las tablas disponibles
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tablas disponibles:")
print(tables)

# Leer los datos de cada tabla
authors = pd.read_sql_query("SELECT * FROM authors;", conn)
comments = pd.read_sql_query("SELECT * FROM comments;", conn)
posts = pd.read_sql_query("SELECT * FROM posts;", conn)
subreddits = pd.read_sql_query("SELECT * FROM subreddits;", conn)

# Visualizar primeras filas de cada tabla
print("\n=== Tabla: authors ===")
display(authors.head())

print("\n=== Tabla: comments ===")
display(comments.head())

print("\n=== Tabla: posts ===")
display(posts.head())

print("\n=== Tabla: subreddits ===")
display(subreddits.head())

# Cerrar la conexión
conn.close()


Tablas disponibles:
         name
0  subreddits
1     authors
2       posts
3    comments

=== Tabla: authors ===


,author_name
0,Typical_Wafer_1324
1,krobzaur
2,ITagEveryone
3,ditlevrisdahl
4,geovane_jeff



=== Tabla: comments ===


,comment_id,post_id,author,body,created_utc,parent_id,link_id,score,is_submitter
0,ng4q0es,1nq1588,Present_Tonight1813,I made a program that prompts the user for a s...,1.758810e+09,t3_1nq1588,t3_1nq1588,2,0
1,ng4cg2y,1nq1588,cptsdemon,I made a tool called [PyLiveDev](https://pypi....,1.758805e+09,t3_1nq1588,t3_1nq1588,2,0
2,ng6dq2g,1nq1588,Fr1dge21,As my first project I managed to automate stoc...,1.758827e+09,t3_1nq1588,t3_1nq1588,4,0
3,ng8r3e5,1nq1588,AdventPriest,"Full disclosure, I've leaned heavily on AI to ...",1.758856e+09,t1_ng4aj2a,t3_1nq1588,1,0
4,ng5ys9z,1nq1588,geovane_jeff,My own backup app :D saves me every week!,1.758822e+09,t3_1nq1588,t3_1nq1588,4,0



=== Tabla: posts ===


,post_id,title,selftext,url,permalink,score,num_comments,over_18,created_utc,link_flair_text,is_self,spoiler,locked,thumbnail,subreddit,author
0,1nqnm44,PEP 806 – Mixed sync/async context managers wi...,PEP 806 – Mixed sync/async context managers wi...,https://www.reddit.com/r/Python/comments/1nqnm...,/r/Python/comments/1nqnm44/pep_806_mixed_synca...,107,19,0,1.758847e+09,News,1,0,0,self,PYTHON,kirara0048
1,1nqfyqh,Looking for Feedback and suggestions: Soundmen...,[Soundmentations](https://github.com/saumyarr8...,https://www.reddit.com/r/Python/comments/1nqfy...,/r/Python/comments/1nqfyqh/looking_for_feedbac...,3,0,0,1.758828e+09,Discussion,1,0,0,self,PYTHON,saumyarr8
2,1nq45ep,migrating from django to FastAPI,We've hit the scaling wall with our decade-old...,https://www.reddit.com/r/Python/comments/1nq45...,/r/Python/comments/1nq45ep/migrating_from_djan...,19,53,0,1.758800e+09,Discussion,1,0,0,self,PYTHON,No-Excitement-7974
3,1nq1588,What small Python automation projects turned o...,I’m trying to level up through practice and I’...,https://www.reddit.com/r/Python/comments/1nq15...,/r/Python/comments/1nq1588/what_small_python_a...,141,87,0,1.758788e+09,Discussion,1,0,0,self,PYTHON,MENTX3
4,1nq5x1b,PyCon AU 2025 talks are all up!,This year's PyCon AU talks have all been uploa...,https://www.reddit.com/r/Python/comments/1nq5x...,/r/Python/comments/1nq5x1b/pycon_au_2025_talks...,19,1,0,1.758805e+09,Resource,1,0,0,self,PYTHON,fphhotchips



=== Tabla: subreddits ===


,subreddit,subscribers,description,created_utc,over18
0,PYTHON,1396681,The official Python community for Reddit! Stay...,1.201231e+09,0
1,FUNNY,66831915,Reddit's largest humor depository,1.201243e+09,0
2,PUBLICFREAKOUT,4730514,"A subreddit dedicated to people freaking out, ...",1.381610e+09,0
3,ASKREDDIT,57220281,r/AskReddit is the place to ask and answer tho...,1.201233e+09,0
4,BALDURSGATE3,3229852,"A community all about Baldur's Gate III, the r...",1.559227e+09,0


In [3]:
# 1. Limpieza mínima: revisar valores faltantes y preparar columnas clave para análisis
comments['has_author'] = comments['author'].notna()

# Conversión de 'created_utc' a datetime para análisis temporal
comments['created_dt'] = pd.to_datetime(comments['created_utc'], unit='s')

# Resumen estructural útil para NLP
display(comments[['author', 'body', 'score', 'is_submitter', 'created_dt']].describe(include='all'))
comments.isna().sum()

# Información estructural
comments.info()

# Conteo de valores nulos
display(comments.isna().sum())

# Proporción de comentarios eliminados ('[deleted]' / '[removed]')
deleted_mask = comments['body'] == '[deleted]'
removed_mask = comments['body'] == '[removed]'

deleted_rate = deleted_mask.mean()
removed_rate = removed_mask.mean()

display({
    "deleted_rate": deleted_rate,
    "removed_rate": removed_rate
})

# Eliminación opcional de comentarios sin contenido útil
comments_clean = comments[~deleted_mask & ~removed_mask].copy()
comments_clean.reset_index(drop=True, inplace=True)


,author,body,score,is_submitter,created_dt
count,4688,4713,4713.000000,4713.000000,4713
unique,3856,4677,NaN,NaN,NaN
top,AndILoveHe,[deleted],NaN,NaN,NaN
freq,15,15,NaN,NaN,NaN
mean,NaN,NaN,56.878421,0.004880,2025-11-13 23:24:29.534478848
min,NaN,NaN,-81.000000,0.000000,2025-09-25 08:38:29
25%,NaN,NaN,1.000000,0.000000,2025-11-11 00:20:19
50%,NaN,NaN,4.000000,0.000000,2025-11-11 06:26:20
75%,NaN,NaN,16.000000,0.000000,2025-11-11 16:21:10
max,NaN,NaN,14829.000000,1.000000,2025-12-02 06:39:20


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4713 entries, 0 to 4712
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   comment_id    4713 non-null   object        
 1   post_id       4713 non-null   object        
 2   author        4688 non-null   object        
 3   body          4713 non-null   object        
 4   created_utc   4713 non-null   float64       
 5   parent_id     4713 non-null   object        
 6   link_id       4713 non-null   object        
 7   score         4713 non-null   int64         
 8   is_submitter  4713 non-null   int64         
 9   has_author    4713 non-null   bool          
 10  created_dt    4713 non-null   datetime64[ns]
dtypes: bool(1), datetime64[ns](1), float64(1), int64(2), object(6)
memory usage: 372.9+ KB


comment_id       0
post_id          0
author          25
body             0
created_utc      0
parent_id        0
link_id          0
score            0
is_submitter     0
has_author       0
created_dt       0
dtype: int64

{'deleted_rate': 0.003182686187141948, 'removed_rate': 0.0014852535539995756}

In [4]:
# 2. Distribución del score (sin agrupación automática de Plotly)
import plotly.express as px

# Histogram sin rangos arbitrarios (nbins auto = granular)
fig = px.histogram(
    comments_clean,
    x='score',
    opacity=0.7,
    title="Distribución del score (sin agrupación)",
    marginal="rug"
)
fig.update_layout(bargap=0.05)
fig.show()

# Boxplot de score correctamente construido (caja + bigotes)
fig = px.box(
    comments_clean,
    y="score",
    points=False,               # No scatter; esto limpia visualmente los bigotes
    title="Boxplot del score"
)
fig.show()

# Cuantiles del score para referencia
display(comments_clean['score'].quantile([0.01,0.25,0.50,0.75,0.99]))


0.01      -1.0
0.25       1.0
0.50       4.0
0.75      16.0
0.99    1134.2
Name: score, dtype: float64

In [5]:
# 3. Longitud del texto para análisis de sentimiento
comments_clean['text_len'] = comments_clean['body'].str.len()
comments_clean['word_count'] = comments_clean['body'].str.split().str.len()

# Histograma sin agrupación
fig = px.histogram(
    comments_clean,
    x='text_len',
    nbins=len(comments_clean['text_len'].unique()), # granular al máximo
    opacity=0.7,
    title="Distribución de longitud del texto (sin agrupación)"
)
fig.update_layout(bargap=0.01)
fig.show()

# Boxplot real y limpio de text_len
fig = px.box(
    comments_clean,
    y="text_len",
    points=False,
    title="Boxplot de longitud del texto"
)
fig.show()

# Resumen estadístico
display(comments_clean[['text_len','word_count']].describe())


,text_len,word_count
count,4691.000000,4691.000000
mean,156.683010,27.581965
std,228.139246,39.511984
min,1.000000,1.000000
25%,42.000000,7.000000
50%,93.000000,16.000000
75%,184.000000,33.000000
max,5576.000000,859.000000


In [8]:
# 4. Indicadores de tono emocional
comments_clean['has_exclamation'] = comments_clean['body'].str.contains('!', regex=False)
comments_clean['has_question'] = comments_clean['body'].str.contains('?', regex=False)

# Barras con proporciones
prop_df = comments_clean[['has_exclamation','has_question']].mean()
prop_df = prop_df.reset_index().rename(columns={'index':'feature',0:'proportion'})

fig = px.bar(
    prop_df,
    x='feature',
    y='proportion',
    title="Proporción de signos emocionales en comentarios"
)
fig.show()

# Boxplot score vs exclamación (bien formado)
fig = px.box(
    comments_clean,
    x='has_exclamation',
    y='score',
    points=False,
    title="Score según presencia de exclamación"
)
fig.show()

display(comments_clean[['has_exclamation','has_question']].describe())
print("Proporción de comentarios con exclamación:", comments_clean['has_exclamation'].mean())
print("Proporción de comentarios con pregunta:", comments_clean['has_question'].mean())


,has_exclamation,has_question
count,4691,4691
unique,2,2
top,False,False
freq,4064,3852


Proporción de comentarios con exclamación: 0.13366020038371348
Proporción de comentarios con pregunta: 0.17885312300149223


In [16]:
# 5. Relación entre score y longitud del comentario
fig = px.scatter(
    comments_clean,
    x="text_len",
    y="score",
    opacity=0.4,
    title="Relación entre longitud del texto y score"
)
fig.show()

# Correlaciones principales
corr = comments_clean[['score','text_len','word_count']].corr()
display(corr)

# Cuartiles de longitud y score con boxplot correctamente formado
comments_clean['len_bucket'] = pd.qcut(
    comments_clean['text_len'],
    q=4,
    labels=['Q1','Q2','Q3','Q4']
)

fig = px.box(
    comments_clean,
    x="len_bucket",
    y="score",
    points=False,
    title="Score por cuartiles de longitud (boxplot correcto)"
)
fig.show()

print("Cuartiles de longitud del texto:")
display(comments_clean['text_len'].quantile([0.25,0.50,0.75]))

print("Cuartiles del score:")
display(comments_clean['score'].quantile([0.25,0.50,0.75]))

print("Correlación entre score y longitud del texto:")
display(comments_clean[['score','text_len']].corr())
display(comments_clean[['score','word_count']].corr())
display(comments_clean[['text_len','word_count']].corr())
display(comments_clean[['score']].describe())


print("Comentarios limpiados:", len(comments_clean), "de", len(comments))
print("Proporción de comentarios retenidos:", len(comments_clean)/len(comments))




,score,text_len,word_count
score,1.000000,-0.004142,-0.002269
text_len,-0.004142,1.000000,0.988565
word_count,-0.002269,0.988565,1.000000


Cuartiles de longitud del texto:


0.25     42.0
0.50     93.0
0.75    184.0
Name: text_len, dtype: float64

Cuartiles del score:


0.25     1.0
0.50     4.0
0.75    16.0
Name: score, dtype: float64

...

,score,text_len
score,1.000000,-0.004142
text_len,-0.004142,1.000000


,score,word_count
score,1.000000,-0.002269
word_count,-0.002269,1.000000


,text_len,word_count
text_len,1.000000,0.988565
word_count,0.988565,1.000000


,score
count,4691.000000
mean,57.125133
std,402.498421
min,-81.000000
25%,1.000000
50%,4.000000
75%,16.000000
max,14829.000000


Comentarios limpiados: 4691 de 4713
Proporción de comentarios retenidos: 0.9953320602588585


In [26]:
# ===========================
# Chunk EXTRA · Ejemplos de sentimientos (proxy basado en score)
# ===========================

# Calcular la mediana EXACTA del score en comments_clean
median_score = comments_clean["score"].median()

# Crear una columna temporal de sentimiento para mostrar ejemplos
comments_clean["sentiment_bin"] = (comments_clean["score"] >= median_score).astype(int)

print(f"Mediana del score: {median_score}")
print("0 = negativo (score < mediana), 1 = positivo (score >= mediana)")
print("\n")

# Ejemplo negativo (sentiment = 0)
example_neg = (
    comments_clean[comments_clean["sentiment_bin"] == 0]
    .sort_values("score", ascending=True)
    .iloc[0][["score", "body"]]
)

# Ejemplo positivo (sentiment = 1)
example_pos = (
    comments_clean[comments_clean["sentiment_bin"] == 1]
    .sort_values("score", ascending=False)
    .iloc[110][["score", "body"]]
)

print("===== Ejemplo de comentario NEGATIVO (score bajo) =====")
display(example_neg)

print("\n===== Ejemplo de comentario POSITIVO (score alto) =====")
display(example_pos)

# Opcional: borrar columna auxiliar
comments_clean.drop(columns=["sentiment_bin"], inplace=True)


Mediana del score: 4.0
0 = negativo (score < mediana), 1 = positivo (score >= mediana)


===== Ejemplo de comentario NEGATIVO (score bajo) =====


score                                     -81
body     Then what barriers is she breaking? 
Name: 347, dtype: object


===== Ejemplo de comentario POSITIVO (score alto) =====


score                                                404
body     Boil the carcass down for broth for soup later!
Name: 4108, dtype: object